In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

%matplotlib inline 
# sets the backend of matplotlib to the 'inline' backend, with this backend, the output of plotting commands is displayed inline directly
# below the code cell that produced it. The resulting plots will then also be stored in the notebook document.


In [ ]:
# Download CIFAR-10 dataset


transform = transforms.Compose(
    [transforms.ToTensor(), # The output of torchvision datasets are PILImage images of range [0, 1]
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))] # We transform them to Tensors of normalized range [-1, 1]
)

train_dataset = CIFAR10(root = './data', train = True, download=True, transform=ToTensor())
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

train_dataset_size = len(train_dataset)
test_dataset_size = len(test_dataset)

#classes = (plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
classes = train_dataset.classes
num_classes = len(classes)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Basic perceptron model
class perceptron(torch.nn.Module):
    def __init__(self):
        super(perceptron, self).__init__()
        self.fc = nn.Linear(1,1)            # y = xA.T + b, input = Nx1, output = Nx1
        self.relu = torch.nn.ReLU()         # element-wise ReLU
    def forward(self, x):
        output = self.fc(x)
        output = self.relu(x)
        return output     

In [ ]:
class Feedforward(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Feedforward, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
        self.relu = torch.nn.ReLU()
        
        self.fc2 = torch.nn.Linear(self.hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x):
        hidden = self.fc1(x)
        relu = self.relu(hidden)
        
        output = self.fc2(relu)
        output = self.sigmoid(output)
        return output

In [ ]:
# create random data points
from sklearn.datasets import make_blobs

def blob_label(y, label, loc): # assign labels
    target = np.copy(y)
    for l in loc:
        target[y == l] = label
    return target

x_train, y_train = make_blobs(n_samples = 40, n_features = 2, cluster_std=1.5, shuffle = True)
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(blob_label(y_train, 0, [0]))
y_train = torch.FloatTensor(blob_label(y_train, 1, [1,2,3]))

x_test, y_test = make_blobs(n_samples = 10, n_features = 2, cluster_std=1.5, shuffle = True)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(blob_label(y_test, 0, [0]))
y_test = torch.FloatTensor(blob_label(y_test, 1, [1,2,3]))

In [ ]:
# model, criterion, optimizer
model = Feedforward(2, 10)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# evaluate model's performance before training

model.eval()
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print("Test loss before training", before_train.item())

Test loss before training 0.7167186141014099


In [ ]:
# switch back to training mode

model.train()
epoch = 500

for epoch in range(epoch):
    optimizer.zero_grad() # set gradient to zero before start of backpropagation, pytorch
                          # accumulates gradients from backward passes of previous epochs
    
    # Forward Pass
    y_pred = model(x_train)
    
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
    print('Epoch: {}, train loss: {}'.format(epoch, loss.item()))
    
    # Backward Pass
    loss.backward() # compute gradients
    optimizer.step() # update weights

Epoch: 0, train loss: 0.7925909757614136
Epoch: 1, train loss: 0.7148780226707458
Epoch: 2, train loss: 0.6482788324356079
Epoch: 3, train loss: 0.591232180595398
Epoch: 4, train loss: 0.5422729849815369
Epoch: 5, train loss: 0.5001434087753296
Epoch: 6, train loss: 0.4635399281978607
Epoch: 7, train loss: 0.43163514137268066
Epoch: 8, train loss: 0.40365567803382874
Epoch: 9, train loss: 0.37900668382644653
Epoch: 10, train loss: 0.3571118712425232
Epoch: 11, train loss: 0.3375438451766968
Epoch: 12, train loss: 0.31998375058174133
Epoch: 13, train loss: 0.30416759848594666
Epoch: 14, train loss: 0.2898276448249817
Epoch: 15, train loss: 0.2767683267593384
Epoch: 16, train loss: 0.2648358941078186
Epoch: 17, train loss: 0.2538939416408539
Epoch: 18, train loss: 0.24382655322551727
Epoch: 19, train loss: 0.23453500866889954
Epoch: 20, train loss: 0.22593474388122559
Epoch: 21, train loss: 0.21795253455638885
Epoch: 22, train loss: 0.2105253040790558
Epoch: 23, train loss: 0.20359805226

In [ ]:
# Test Mode again

model.eval()
y_pred = model(x_test)
after_train = criterion(y_pred.squeeze(), y_test)
print('Test loss after Training:', after_train.item())

Test loss after Training: 1.8983100652694702
